In [16]:
%pylab inline

import gc
import fpfs
import numpy as np
from fpfs import simutil
import matplotlib.pylab as plt
import astropy.io.fits as pyfits
import numpy.lib.recfunctions as rfn
import pdet
import pltutil

Populating the interactive namespace from numpy and matplotlib


/lustre/work/xiangchong.li/ana/miniconda3/envs/cosmosis-mpich/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['beta', 'plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [23]:
ishear =1
igroup =0
do_plot=False
ngrid  =64
ngal=  1
img1=  simutil.make_basic_sim('basicCenter_psf60','g%d-0000' %ishear,0,ny=ngal,nx=ngal,do_write=False)
img2=  simutil.make_basic_sim('basicCenter_psf60','g%d-2222' %ishear,0,ny=ngal,nx=ngal,do_write=False)
ngrid2=ngrid*ngal
beta=  0.75
gsigma =   6.*2.*np.pi/64


# PSF
psf     =   pyfits.getdata('../sim/galaxy_basic_psf60/psf-60.fits')
npad    =   (ngrid2-psf.shape[0])//2
psfData =   np.pad(psf,(npad+1,npad),mode='constant')
assert psfData.shape[0]==ngrid2

rcut    =   16
beg     =   ngrid//2-rcut
end     =   beg+2*rcut
psfData2=   psfData[beg:end,beg:end]

noiVar      =   4e-3
noiFname    =   os.path.join('../sim/noise','noi0000.fits')
# multiply by 10 since the noise has variance 0.01
noiData     =   pyfits.open(noiFname)[0].data*10.
# same for the noivar model
powIn       =   np.load('../sim/corPre/noiPows2.npy',allow_pickle=True).item()['%s'%rcut]*noiVar*100
powModel    =   np.zeros((1,powIn.shape[0],powIn.shape[1]))
powModel[0] =   powIn
fpTask      =   fpfs.fpfsBase.fpfsTask(psfData2,noiFit=powModel[0],beta=beta)

indX    =   np.arange(32,64,64)
indY    =   np.arange(32,64,64)
inds    =   np.meshgrid(indY,indX,indexing='ij')

coords  =   np.array(np.zeros(inds[0].size),dtype=[('pdet_y','i4'),('pdet_x','i4')])
coords['pdet_y']=   np.ravel(inds[0])
coords['pdet_x']=   np.ravel(inds[1])
del indX,indY,inds

In [43]:
fpTask0 =   fpfs.fpfsBase.fpfsTask(psfData2,beta=beta)
# noiseless measurement
imgList =   [img1[cc['pdet_y']-rcut:cc['pdet_y']+rcut,\
            cc['pdet_x']-rcut:cc['pdet_x']+rcut] for cc in coords]
o       =   fpTask0.measure(imgList)
ell0    =   fpfs.fpfsBase.fpfsM2E(o,const=100)
out0    =   pdet.get_shear_response(img1,psfData,gsigma=gsigma,\
            coords=coords)
del imgList,o
gc.collect()

0

In [44]:
dAll1=[]
for j in range(1):
    for i in range(1):
        imn=img1+noiData[j*64:(j+1)*64,i*64:(i+1)*64]
        imgList =   [imn[cc['pdet_y']-rcut:cc['pdet_y']+rcut,\
                    cc['pdet_x']-rcut:cc['pdet_x']+rcut] for cc in coords]
        o       =   fpTask0.measure(imgList)
        ell     =   fpfs.fpfsBase.fpfsM2E(o,const=100)
        out     =   pdet.get_shear_response(imn,psfData,gsigma=gsigma,\
                    coords=coords)
        dAll1.append(o['fpfs_e1']*o1['pdet_v22r1'])
        del o

ValueError: no field of name fpfs_e1

In [41]:
o

array((22371.82451703, 1089.36514904, 7799.07635582, 157467.20370865),
      dtype=[('fpfs_M00', '>f8'), ('fpfs_M22c', '>f8'), ('fpfs_M22s', '>f8'), ('fpfs_M40', '>f8')])